In [ ]:

from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import json
import re

url = 'https://www.aonprd.com/'

class ArchetypeScraper:
    def __init__(self, url):
        self.url = url
        self.href = f'Races.aspx?Category=Core'
        self.page = requests.get(url + self.href)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        self.table = self.soup.find("table")
        self.href2_list = []

    def clean_text(self, text):
        # Replace Unicode right single quotation mark with an apostrophe
        cleaned_text = [line.replace('\u2019', "'").replace('\u2018', "'").replace('\u201c', '"').replace('\u201d', '"') for line in text.split('\n') if line.strip() and line.strip() != "."]
        return ' '.join(cleaned_text).strip()
    
    def remove_parenthesis(self, input_string):
        pattern = r"\(.*?pg\..*?\)"
        result = re.sub(pattern, '', input_string)
        result = re.sub(r'<.*?>', '', result)        

        #this removes parenthesis with only text in them 
        result = re.sub(r'\(\d+\)', '', result)

        return result

        

    def link_searcher(self, a_tags):
        for a in a_tags:
            # Check if the link contains "investigator-talents" in the href attribute
            if "RacesDisplay.aspx?ItemName=" in str(a.get('href', '')):
                # Extract everything within double quotes in the href attribute
                regex = r'href="(.*?)"'
                href_unclean = str(a)
                match = re.search(regex, href_unclean)

                if match:
                    self.href2 = match.group(1)
                    self.href2_list.append(self.href2)
                else:
                    continue
            else:
                continue    


        return self.href2_list

         

# What we want to do is to get the data once we see a <b> tag, use the <b> tag as the key section
# Since it's going to be the same each time we can manually set them (like in the items json file)
    def div_search(self, td_tags):
        
        result_list = []

        for td in td_tags:
            a_tags = td.find_all('a')
            data_dict = {}            
            href2_list = self.link_searcher(a_tags)

        
    def talent_info_search(self):
        # Initialize an empty list to store dictionaries for each iteration
        all_data_dicts = []
        h2_tag_list = []
        h3_tag_list = []
        all_h_tags = []

        print(self.href2_list)

        for i, href2 in enumerate(self.href2_list):
            self.detail_page = requests.get(url + href2)
            self.detail_soup = BeautifulSoup(self.detail_page.content, 'html.parser')

            b_tags = self.detail_soup.find_all('b')
            print(b_tags)

            h2_tags = self.detail_soup.find_all('h2')
            h3_tags = self.detail_soup.find_all('h3')

            for h2 in h2_tags:
                h2_tag = self.remove_parenthesis(h2.get_text())
                if 'Archive' in h2_tag or 'Aspects' in h2_tag:
                    continue
                else:
                    print(h2_tag)

            for h3 in h3_tags:
                h3_tag = self.remove_parenthesis(h3.get_text())
                if 'Archive' in h3_tag or 'Aspects' in h3_tag:
                    continue
                else:
                    print(h3_tag)

            data_dict = {}


            for b_tag in b_tags:
                # Find the next_sibling until we hit the </br> tag
                next_sibling = b_tag.next_sibling
                text_under_b = ""

                while next_sibling and (not isinstance(next_sibling, Tag) or next_sibling.name not in ['br']):
                    if isinstance(next_sibling, Tag):
                        if next_sibling.name == 'a':
                            text_under_b += self.clean_text(next_sibling.get_text(strip=True))
                        elif next_sibling.name == 'br':
                            break
                    else:
                        text_under_b += self.clean_text(next_sibling.strip())
                    
                    next_sibling = next_sibling.next_sibling


                if text_under_b:
                    key = self.clean_text(b_tag.get_text(strip=True))
                    key = self.remove_parenthesis(key)
                    value = self.clean_text(text_under_b.strip())
                    value = self.remove_parenthesis(value)
                    data_dict[key] = value            

            for h_tag in h3_tags:
                print(f'This is your h_tag: {h_tag}')
                
                # Find the next_sibling until we hit the </h3> tag
                next_sibling = h_tag.next_sibling
                text_under_h = ""
                extract_text = False  # Flag to indicate whether to extract text

                while next_sibling and (not isinstance(next_sibling, Tag) or next_sibling.name not in ['h3']):
                    if isinstance(next_sibling, Tag):
                        if next_sibling.name == 'a':
                            skip = True
                        elif next_sibling.name == 'br':
                            extract_text = True
                    else:
                        if extract_text:
                            text_under_h += str(next_sibling).strip()
                    
                    next_sibling = next_sibling.next_sibling

                if text_under_h:
                    key = h_tag.get_text(strip=True)
                    value = text_under_h.strip()
                    
                    # Check if the key already exists in 'Subraces'
                    if 'Subraces' not in data_dict:
                        data_dict['Subraces'] = {}
                    
                    # Check if the key already exists for the specific 'Subraces' entry
                    if key not in data_dict['Subraces']:
                        data_dict['Subraces'][key] = []
                    
                    # Append the new value to the list under the specific key
                    data_dict['Subraces'][key].append(value)

            for h_tag in h2_tags:
                print(f'This is your h_tag: {h_tag}')
                
                # Find the next_sibling until we hit the </h3> tag
                next_sibling = h_tag.next_sibling
                text_under_h = ""
                extract_text = False  # Flag to indicate whether to extract text

                while next_sibling and (not isinstance(next_sibling, Tag) or next_sibling.name not in ['h3']):
                    if isinstance(next_sibling, Tag):
                        if next_sibling.name == 'a':
                            skip = True
                        elif next_sibling.name == 'br':
                            extract_text = True
                    else:
                        if extract_text:
                            text_under_h += str(next_sibling).strip()
                    
                    next_sibling = next_sibling.next_sibling

                if text_under_h:
                    key = h_tag.get_text(strip=True)
                    value = text_under_h.strip()
                    
                    # Check if the key already exists in 'Replacements'
                    if 'Replacements' not in data_dict:
                        data_dict['Replacements'] = {}
                    
                    # Check if the key already exists for the specific 'Replacements' entry
                    if key not in data_dict['Replacements']:
                        data_dict['Replacements'][key] = []
                    
                    # Append the new value to the list under the specific key
                    data_dict['Replacements'][key].append(value)
              

            title = self.grab_title(href2)
            all_data_dicts.append({title: data_dict})
            # Convert the list of dictionaries to JSON format and print it
            json_output = json.dumps(all_data_dicts, indent=2)
            with open('CoreRaces.json', 'w', encoding='utf-8') as json_file:
                json.dump(all_data_dicts, json_file, indent=2, ensure_ascii=False)            
            print(json_output)

    def main(self):        
        td_tags = self.soup.find_all('td')
        self.div_search(td_tags)
        self.talent_info_search()

    def grab_title(self, href):
        pattern = r'=(.*)$'
        match = re.search(pattern, href)

        if match:
            result = match.group(1)
        return result


# Instantiate the ArchetypeScraper class
scraper_instance = ArchetypeScraper(url)

# Call the get_class_info method
scraper_instance.main()

In [ ]:

from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import json
import re

url = 'https://www.aonprd.com/'

class ArchetypeScraper:
    def __init__(self, url):
        self.url = url
        self.href = f'Races.aspx?Category=NonCore'
        self.page = requests.get(url + self.href)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        self.table = self.soup.find("table")
        self.href2_list = []

    def clean_text(self, text):
        # Replace Unicode right single quotation mark with an apostrophe
        cleaned_text = [line.replace('\u2019', "'").replace('\u2018', "'").replace('\u201c', '"').replace('\u201d', '"') for line in text.split('\n') if line.strip() and line.strip() != "."]
        return ' '.join(cleaned_text).strip()
    
    def remove_parenthesis(self, input_string):
        pattern = r"\(.*?pg\..*?\)"
        result = re.sub(pattern, '', input_string)
        result = re.sub(r'<.*?>', '', result)        

        #this removes parenthesis with only text in them 
        result = re.sub(r'\(\d+\)', '', result)

        return result

        

    def link_searcher(self, a_tags):
        for a in a_tags:
            # Check if the link contains "investigator-talents" in the href attribute
            if "RacesDisplay.aspx?ItemName=" in str(a.get('href', '')):
                # Extract everything within double quotes in the href attribute
                regex = r'href="(.*?)"'
                href_unclean = str(a)
                match = re.search(regex, href_unclean)

                if match:
                    self.href2 = match.group(1)
                    self.href2_list.append(self.href2)
                else:
                    continue
            else:
                continue    


        return self.href2_list

         

# What we want to do is to get the data once we see a <b> tag, use the <b> tag as the key section
# Since it's going to be the same each time we can manually set them (like in the items json file)
    def div_search(self, td_tags):
        
        result_list = []

        for td in td_tags:
            a_tags = td.find_all('a')
            data_dict = {}            
            href2_list = self.link_searcher(a_tags)

        
    def talent_info_search(self):
        # Initialize an empty list to store dictionaries for each iteration
        all_data_dicts = []
        h2_tag_list = []
        h3_tag_list = []
        all_h_tags = []

        print(self.href2_list)

        for i, href2 in enumerate(self.href2_list):
            self.detail_page = requests.get(url + href2)
            self.detail_soup = BeautifulSoup(self.detail_page.content, 'html.parser')

            b_tags = self.detail_soup.find_all('b')
            print(b_tags)

            h2_tags = self.detail_soup.find_all('h2')
            h3_tags = self.detail_soup.find_all('h3')

            for h2 in h2_tags:
                h2_tag = self.remove_parenthesis(h2.get_text())
                if 'Archive' in h2_tag or 'Aspects' in h2_tag:
                    continue
                else:
                    print(h2_tag)

            for h3 in h3_tags:
                h3_tag = self.remove_parenthesis(h3.get_text())
                if 'Archive' in h3_tag or 'Aspects' in h3_tag:
                    continue
                else:
                    print(h3_tag)

            data_dict = {}


            for b_tag in b_tags:
                # Find the next_sibling until we hit the </br> tag
                next_sibling = b_tag.next_sibling
                text_under_b = ""

                while next_sibling and (not isinstance(next_sibling, Tag) or next_sibling.name not in ['br']):
                    if isinstance(next_sibling, Tag):
                        if next_sibling.name == 'a':
                            text_under_b += self.clean_text(next_sibling.get_text(strip=True))
                        elif next_sibling.name == 'br':
                            break
                    else:
                        text_under_b += self.clean_text(next_sibling.strip())
                    
                    next_sibling = next_sibling.next_sibling


                if text_under_b:
                    key = self.clean_text(b_tag.get_text(strip=True))
                    key = self.remove_parenthesis(key)
                    value = self.clean_text(text_under_b.strip())
                    value = self.remove_parenthesis(value)
                    data_dict[key] = value            

            for h_tag in h3_tags:
                print(f'This is your h_tag: {h_tag}')
                
                # Find the next_sibling until we hit the </h3> tag
                next_sibling = h_tag.next_sibling
                text_under_h = ""
                extract_text = False  # Flag to indicate whether to extract text

                while next_sibling and (not isinstance(next_sibling, Tag) or next_sibling.name not in ['h3']):
                    if isinstance(next_sibling, Tag):
                        if next_sibling.name == 'a':
                            skip = True
                        elif next_sibling.name == 'br':
                            extract_text = True
                    else:
                        if extract_text:
                            text_under_h += str(next_sibling).strip()
                    
                    next_sibling = next_sibling.next_sibling

                if text_under_h:
                    key = h_tag.get_text(strip=True)
                    value = text_under_h.strip()
                    
                    # Check if the key already exists in 'Subraces'
                    if 'Subraces' not in data_dict:
                        data_dict['Subraces'] = {}
                    
                    # Check if the key already exists for the specific 'Subraces' entry
                    if key not in data_dict['Subraces']:
                        data_dict['Subraces'][key] = []
                    
                    # Append the new value to the list under the specific key
                    data_dict['Subraces'][key].append(value)

            for h_tag in h2_tags:
                print(f'This is your h_tag: {h_tag}')
                
                # Find the next_sibling until we hit the </h3> tag
                next_sibling = h_tag.next_sibling
                text_under_h = ""
                extract_text = False  # Flag to indicate whether to extract text

                while next_sibling and (not isinstance(next_sibling, Tag) or next_sibling.name not in ['h3']):
                    if isinstance(next_sibling, Tag):
                        if next_sibling.name == 'a':
                            skip = True
                        elif next_sibling.name == 'br':
                            extract_text = True
                    else:
                        if extract_text:
                            text_under_h += str(next_sibling).strip()
                    
                    next_sibling = next_sibling.next_sibling

                if text_under_h:
                    key = h_tag.get_text(strip=True)
                    value = text_under_h.strip()
                    
                    # Check if the key already exists in 'Replacements'
                    if 'Replacements' not in data_dict:
                        data_dict['Replacements'] = {}
                    
                    # Check if the key already exists for the specific 'Replacements' entry
                    if key not in data_dict['Replacements']:
                        data_dict['Replacements'][key] = []
                    
                    # Append the new value to the list under the specific key
                    data_dict['Replacements'][key].append(value)
              

            title = self.grab_title(href2)
            all_data_dicts.append({title: data_dict})
            # Convert the list of dictionaries to JSON format and print it
            json_output = json.dumps(all_data_dicts, indent=2)
            with open('NonCoreRaces.json', 'w', encoding='utf-8') as json_file:
                json.dump(all_data_dicts, json_file, indent=2, ensure_ascii=False)

            print(json_output)

    def main(self):        
        td_tags = self.soup.find_all('td')
        self.div_search(td_tags)
        self.talent_info_search()

    def grab_title(self, href):
        pattern = r'=(.*)$'
        match = re.search(pattern, href)

        if match:
            result = match.group(1)
        return result


# Instantiate the ArchetypeScraper class
scraper_instance = ArchetypeScraper(url)

# Call the get_class_info method
scraper_instance.main()